In [10]:
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
html = urlopen('http://en.wikipedia.org/wiki/Comparison_of_text_editors')
bs = BeautifulSoup(html, 'html.parser')
# 두 개의 테이블 중에 첫 번째 테이블 사용
table = bs.find_all('table', {'class':'wikitable'})[0]
rows = table.find_all('tr') # tr 요소 : 테이블의 한 줄 한 줄을 가리킴


test = pd.DataFrame([value.text.strip() for row in rows for value in row.find_all(['th','td'])])

# try:
#     for row in rows:
#         csvRow = []
#         for cell in row.find_all(['th', 'td']) : 
#             print(cell.text.strip())
#             csvRow.append(cell.text.strip())
#         writer.writerow(csvRow)
# finally:
#     csvFile.close()

In [11]:
test

,0
0,Name
1,Developer
2,Initial release
3,Latest release
4,Programming language
...,...
696,C
697,No cost
698,GPL-2.0-or-later
699,


In [12]:
import pymysql
conn = pymysql.connect(host='localhost',
user='root', passwd='1234', db='scraping', charset='utf8')
cur = conn.cursor()
cur.execute('use scraping')
cur.execute("SELECT * FROM pages WHERE id=2")
print(cur.fetchone())
cur.close()
conn.close()

(2, 'A new title', 'Some new content', datetime.datetime(2024, 2, 13, 10, 45, 52))


In [13]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime
import random
import pymysql
import re
conn = pymysql.connect(host='localhost',
user='root', passwd='1234', db='scraping', charset='utf8')

cur = conn.cursor()
random.seed(None)
def store(title, content):
    cur.execute('INSERT INTO pages (title, content) VALUES ("%s", "%s")', (title, content))
    cur.connection.commit()
    
def getLinks(articleUrl):
    html = urlopen('http://en.wikipedia.org' + articleUrl)
    bs = BeautifulSoup(html, 'html.parser')
    title = bs.find('h1').text
    content = bs.find('div', {'id':'mw-content-text'}).find('p').text
    # find()로 검색된 데이터를 데이터베이스에 저장
    store(title, content)
    return bs.find('div', {'id':'bodyContent'}).\
        find_all('a', href=re.compile('^(/wiki/)((?!:).)*$'))

links = getLinks('/wiki/Kevin_Bacon')
try:
    while len(links) > 0:
        newArticle = links[random.randint(0, len(links)-1)].attrs['href']
        print(newArticle)
        links = getLinks(newArticle)
finally:
    cur.close()
    conn.close()

/wiki/Tremors_(1990_film)
/wiki/Los_Angeles_Times
/wiki/Gene_Sherman_(reporter)
/wiki/Oak_Park,_Illinois
/wiki/Fallingwater
/wiki/George_Antheil
/wiki/IRCAM
/wiki/Future_Radio
/wiki/Restricted_Service_Licence
/wiki/Sky_Sports
/wiki/Kris_Boyd
/wiki/Football_League_Championship
/wiki/1947%E2%80%9348_Football_League#Second_Division
/wiki/Accrington_Stanley_F.C._(1891)
/wiki/Accrington_power_station
/wiki/Sellafield
/wiki/Drigg
/wiki/Borough_of_Copeland
/wiki/Frizington
/wiki/Member_of_parliament


KeyboardInterrupt: 